## What is a Sudoku?

[Sudoku](https://en.wikipedia.org/wiki/Sudoku) is one of the world's most popular puzzles. It consists of a 9x9 grid, and the objective is to fill the grid with digits in such a way that each row, each column, and each of the 9 principal 3x3 subsquares contains all of the digits from 1 to 9. The detailed rules can be found, for example, [here](https://www.conceptispuzzles.com/index.aspx?uri=puzzle/sudoku/rules).

The puzzle is given as partially completed grid, and the goal is to fill in the missing numbers. Below is an example of such grid. 

![alt-text](./images/sudoku-easy.png)

## Goal of this project

The main goal of this project is to build an itelligent agent that will solve every sudodu while introducing you to two powerful techniques that are used throughout the field of AI:

- **Constraint Propagation**

    When trying to solve a problem, you'll find that there are some local constraints to each square. These constraints help you narrow the possibilities for the answer, which can be very helpful. We will learn to extract the maximum information out of these constraints in order to get closer to our solution. Additionally, you'll see how we can repeatedly apply simple constraints to iteratively narrow the search space of possible sulutions. Constraint propagation can be used to solve a variety of problems such as calendar scheduling, and cryptographic puzzles.
- **Search**

    In the process of problem solving, we may get to the point where two or more possiblilities are available. What do we do? What if we branch out and consider both of them? Maybe one of them will lead us to a position in which three or more possibilities are available. Then we cana branch out again. At the end, we can create a whole tree of possibilities and find ways to traverse the tree until we find our solution. This is an example of how search can be used.

These ideas may seem simple and they're actually intended to be! Through this lesson you'll see how AI is really composed of very simple ideas can can but put together to solve complex problems. Throughout this lesson, we challenge you to think of how you can apply these ideas to build AI agents to solve other puzzles and problems in your worlds.


# Naming Conventions

## Rows and Columns

Since we're writing an agent to solve the Sudoku puzzle, let's start by laballing rows and columns.

- The rows will be labelled by the letters A, B, C, D, E, F, G, H, I
- The columns will be labelled by the numbers 1, 2, 3, 4, 5, 6, 7, 8, 9. Here we can see the unsolved and solve puzles with the labels for the rows and columns.
- The 3x3 squares won't be labelled, but in the diagram, they can be seen with alternating colors of grey and white.

![](./images/labels.png)

## Boxes, Units and Peers

And let's start naming the important elements created by these rows and columns that are relevant to solving a Sudoku:
- The individual squares at the intersection of rows and columns will be called `boxes`. These boxes will have labels 'A1', 'A2', ..., 'I9'.
- The complete rows, columns, and 3x3 squares will be called `units`. Thus, each unit is a set of 9 boxes, and there are 27 units in total.
- For a particular box (such as 'A1'), its `peers` will be all other boxes that belong to a common unit (namely, those that belong to the same row, colun, or 3x3 square).

Let's see an example. In the grids below, the set of highlighted boxes represent `units`. Each grid shows a different `peer` of the box at E3.

![](./images/peers.png)

# Encoding the Board

![](./images/labels.png)


Now, in order to implement an agent, let's start by coding the board in Python. Then, we'll code the necessary functions to solve the Sudoku. We'll record the puzzles in two ways — as a `string` and as a `dictionary`.

The string will consist of a concatenation of all the readings of the digits in the rows, taking the rows from top to bottom. If the puzzle is not solved, we can use a `.` as a placeholder for an empty box.

For example, the unsolved puzzle at the above left will be written as:
`..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..`

And the solved puzzle at the above right, will be recorded as:
`483921657967345821251876493548132976729564138136798245372689514814253769695417382`

We'll implement the dictionary as follows. The keys will be string corresponding to the boxes - namely, `A1`, `A2`, ..., `I9`. The values will either be the digit in each box (if there is one) or a `'.'` (if not).

So, let's get started. First we'll record rows and columns as strings.

In [1]:
rows = 'ABCDEFGHI'
cols = '123456789'

We'll start by writing a help function, `cross(a, b)`, which given two strings — `a` and `b` — will return the list formed by all the possible concatenations of a letter `s` in string `a` with a letter `t` in string `b`.

So `cross('abc', 'def')` will return `['ad', 'ae', 'af', 'bd', 'be', 'bf', 'cd', 'ce', 'cf']`

In [2]:
def cross(a, b):
    return [s+t for s in a for t in b]

Now, to create the labels of the boxes:

In [3]:
boxes = cross(rows, cols)
print(boxes)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']


And for the units and peers:

In [4]:
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)
peers

{'A1': {'A2',
  'A3',
  'A4',
  'A5',
  'A6',
  'A7',
  'A8',
  'A9',
  'B1',
  'B2',
  'B3',
  'C1',
  'C2',
  'C3',
  'D1',
  'E1',
  'F1',
  'G1',
  'H1',
  'I1'},
 'A2': {'A1',
  'A3',
  'A4',
  'A5',
  'A6',
  'A7',
  'A8',
  'A9',
  'B1',
  'B2',
  'B3',
  'C1',
  'C2',
  'C3',
  'D2',
  'E2',
  'F2',
  'G2',
  'H2',
  'I2'},
 'A3': {'A1',
  'A2',
  'A4',
  'A5',
  'A6',
  'A7',
  'A8',
  'A9',
  'B1',
  'B2',
  'B3',
  'C1',
  'C2',
  'C3',
  'D3',
  'E3',
  'F3',
  'G3',
  'H3',
  'I3'},
 'A4': {'A1',
  'A2',
  'A3',
  'A5',
  'A6',
  'A7',
  'A8',
  'A9',
  'B4',
  'B5',
  'B6',
  'C4',
  'C5',
  'C6',
  'D4',
  'E4',
  'F4',
  'G4',
  'H4',
  'I4'},
 'A5': {'A1',
  'A2',
  'A3',
  'A4',
  'A6',
  'A7',
  'A8',
  'A9',
  'B4',
  'B5',
  'B6',
  'C4',
  'C5',
  'C6',
  'D5',
  'E5',
  'F5',
  'G5',
  'H5',
  'I5'},
 'A6': {'A1',
  'A2',
  'A3',
  'A4',
  'A5',
  'A7',
  'A8',
  'A9',
  'B4',
  'B5',
  'B6',
  'C4',
  'C5',
  'C6',
  'D6',
  'E6',
  'F6',
  'G6',
  'H6',
  'I6'}

## Implement `grid_values()`

**A function to convert the string representation of a puzzle into a dictionary from.**


Recall that for the string:

`
'..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
`

...we'd like to return the dictionary:

```
{
    'A1': '.',
    'A2': '.',
    'A3': '3',
    ...
    'I9': '.'
}
```

In [5]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.
    
    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form.
    """
    return dict(zip(boxes, grid))

### Implement `display()`
**A function to display sudoku board**

In [6]:
def display(values):
    """Display a sudoku board
    Args:
        values: Sudoku in dictionary form.
    """
    width = max(len(s) for _, s in values.items()) + 1
    line = '+'.join(['-'*3*width]*3)
    for r in rows:
        print(''.join(values[r+c].center(width) + ('|' if c in '36' else '') for c in cols ) )
        if r in 'CF':
            print(line)

In [7]:
# Sanity check
values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
display(values)

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


## Strategy 1: Elimination
**Insight: We can eliminate possible values for a box by looking at its peers**

First things first, let's look at a box and analyze the values that could go in there.
![](./images/reduce-values.png)

The possible values are 4 and 7. What did we do to figure this out? Well, all the other values already appear either in the same column, row, or 3x3 square of the highlighted box, so we conclude that they can't be a value for this box. In other words, we used the following strategy.

## Improved `grid_value()`

As of now, we are recording the puzzles in dictionary form, where the keys are the boxes (`'A1', 'A2', ..., 'I9')` and the values are either the value of each box (if a value exists) or `'.'`(if the box has no value assigned yet). What we really want is for each value to represent all the available values for that box. For example, the box in the second row and fifth column above will have key `'B5'` and value `'47'`(because `4` and `7` are the only possible values for it). The staring value for every empty box will thus be `'123456789'`.

Update the `grid_values()` function to return `'123456789'` instead of `'.'` for empty boxes.

In [8]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dictionary with '123456789' value for empties.
    
    Args:
        grid: Sudoku grid in string from, 81 characters long
    Returns:
        Sudoku grid in dictionary form
    """
    sudoku = {}
    for i, v in enumerate(grid):
        sudoku[boxes[i]] = '123456789' if v == '.' else v
    return sudoku

In [9]:
# Sanity check
display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

## Implement eliminate()

Now, let's finish the code for the function `eliminate()`, which will take as input a puzzle in dictionary form. The function will iterate over all the boxes in the puzzle that only have one value assigned to them, and it will remove this value from every one of its peers.

In [10]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.
    
    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.
    
    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    for k, v in values.items():
        if len(v) == 1:
            for pk in peers[k]:
                values[pk] = values[pk].replace(v, '')
    return values

In [11]:
values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
# peers
values = eliminate(values)
display(values)
# values

   45    4578    3   |   9      2      14  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |   59   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6      8      9   |   5     1478    47  
   8     1467    47  |   2      5      3   |   17    1467    9   
   6      69     5   |   4      1      7   |   3     268     26  


The board should looks like this after running `eliminate()`
![](images/values-easy.png)

## Strategy 2: Only Choice

The images above is the values after we apply the function `eliminate`.
Let's actually look more carefully at the top 3x3 square in the center, highlighted in red.

![](./images/highlighted-unit.png)

In this unit, there seems to be only one box which allow a value of 1. Since each digit must appear somewhere in the unit, we conclude that top right box must contain the digit 1.

## Implement `only_choice`

`only_choice` takes a puzzle in dictionary form as input. The function will go through all the units, and if there is a unit with a digit that only gits in one possible box, it will assign that digit to that box.

![](./images/only-choice.png)

In [12]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.
    
    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.
    
    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    
    for unit in unitlist:
        for d in '123456789':
            pos = [box for box in unit if d in values[box]]
            if len(pos) == 1:
                values[pos[0]] = d
    return values

## Apply Constraint Propagation to Sudoku

Combine the function `eliminate` and `only_choice` to write the function `reduce_puzzle`, which receives as input an unsolved puzzle and applies our two constraints repeatedly in an attempt to solve it.

In [13]:
def reduce_puzze(values):
    stalled = False;
    while (not stalled):
        solved_values_before = len([v for v in values.values() if len(v) == 1])
        
        values = eliminate(values)
        values = only_choice(values)
        
        solved_values_after = len([v for v in values.values() if len(v) == 1])
        stalled = solved_values_after == solved_values_before
        
        if len([v for v in values.values() if len(v) == 0]) > 0:
            return False
    return values

In [14]:
# Sanity check
values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
# peers
values = reduce_puzze(values)
display(values)

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


You should got this answer.

![](./images/easy-solution.png)

## Harder Sudoku

Ok, let's see if our algorithm will work all the time. Here's a harder sudoku puzzle:

![](./images/harder-puzzle.png)

In [15]:
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid2)
display(reduce_puzze(values))

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


When we apply `reduce_puzzle`, we get the following grid:

![](images/harder-sudoku-reduced.png)

## Strategy 3: Search

**Pick a box with minimal number of possible values. Try to solve each of the puzzles obtained by choosing each of these values, recursively.**


In [16]:
def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    
    values = reduce_puzze(values)
    if values is False:
        return False
    
    unsolved_boxes = [(len(v), k) for k, v in values.items() if len(v) > 1]
    if len(unsolved_boxes) == 0:
        return values
    
    _, key = min(unsolved_boxes)
    
    for d in values[key]:
        values_copy = values.copy()
        values_copy[key] = d
        returned_value = search(values_copy)
        if returned_value:
            return returned_value
    return False

In [17]:
values = grid_values(grid2)
display(search(values))

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 
